In [1]:
!pip install transformers elasticsearch


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 8.2/8.2 MB 2.7 MB/s eta 0:00:00
     -------------------------------------- 412.8/412.8 kB 2.9 MB/s eta 0:00:00
     -------------------------------------- 330.3/330.3 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 277.5/277.5 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 59.8/59.8 kB 3.3 MB/s eta 0:00:00
     -------------------------------------- 169.0/169.0 kB 2.0 MB/s eta 0:00:00


!pip install torch

In [29]:
!pip install pytorch

  DEPRECATION: pytorch is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  error: subprocess-exited-with-error
  
  × Running setup.py install for pytorch did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\navin.v.i.ASSYST-COC\AppData\Local\Temp\pip-install-zwqod3k0\pytorch_25bcfd20170d4868b95dff11e6d03ff7\setup.py", line 11, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess,


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for pytorch: started
  Running setup.py install for pytorch: finished with status 'error'


# set up libraries

In [31]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
from elasticsearch import Elasticsearch
import torch

ModuleNotFoundError: No module named 'pytorch'

# Define Elasticsearch connection with credentials

# Define the mapping for the dense vector field

In [17]:
mapping = {'properties': {
'embedding': {
'type': 'dense_vector',
'dims': 768 # the number of dimensions of the dense vector
}
}
}

# Create an index with the defined mapping

In [4]:
import requests
requests.packages.urllib3.disable_warnings()
es = Elasticsearch(
    ['https://rag-search-using-elastic.es.us-central1.gcp.cloud.es.io'],
    basic_auth=('elastic', 'y0Wjxx5nEPyPyt8l07pDaTns'),
    verify_certs=False
    #ca_certs="A2525B64D8BFD084D946539261844AC9A3F7DBDC.crt"
)

c:\Users\navin.v.i.ASSYST-COC\AppData\Local\Programs\Python\Python311\Lib\site-packages\elasticsearch\_sync\client\__init__.py:397: SecurityWarning: Connecting to 'https://rag-search-using-elastic.es.us-central1.gcp.cloud.es.io:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [ ]:
es.indices.create(index='chapter-2', body={'mappings': mapping})

In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
from elasticsearch import Elasticsearch
import torch

# Define a set of documents
docs = [
{
'title': 'Document 1',
'text': 'This is the first document.'
},
{
'title': 'Document 2',
'text': 'This is the second document.'
},
{
'title': 'Document 3',
'text': 'This is the third document.'
}
]
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

c:\Users\navin.v.i.ASSYST-COC\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
model.safetensors: 100%|██████████| 440M/440M [03:05<00:00, 2.37MB/s] 
c:\Users\navin.v.i.ASSYST-COC\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\navin.v.i.ASSYST-COC\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#li

In [2]:
# Generate embeddings for the documents using BERT
for doc in docs:
    text = doc['text']
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**inputs).last_hidden_state.mean(dim=1).squeeze(0).numpy()
        doc['embedding'] = output.tolist()

In [6]:
# Index the documents in Elasticsearch
for doc in docs:  es.index(index='chapter-2', body=doc)

In [7]:
#index the verctors so that kNN search is properly done
# first create the mapping
mapping = {
'properties': {
'embedding': {
'type': 'dense_vector',
'dims': 768,
'index': 'true',
"similarity": "cosine"
}
}
}
es.indices.create(index='jokes-index', body={'mappings': mapping})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'jokes-index'})

In [8]:
jokes = [ 
    { 
        'text': 'Why do cats make terrible storytellers? Because they only have one tail.', 
        'category': 'cat' 
    }, 
    { 
        'text': 'What did the cat say when he lost all his money? I am paw.', 
        'category': 'cat' 
    }, 
    { 
        'text': 'Why don\'t cats play poker in the jungle? Too many cheetahs.', 
        'category': 'cat' 
    },
    { 
        'text': 'Why did the tomato turn red? Because it saw the salad dressing!', 
        'category': 'vegetable' 
    },
    { 
        'text': 'Why did the scarecrow win an award? Because he was outstanding in his field.', 
        'category': 'farm' 
    },
    { 
        'text': 'Why did the hipster burn his tongue? Because he drank his coffee before it was cool.', 
        'category': 'hipster' 
    },    
    {
        'text': 'Why did the tomato turn red? Because it saw the salad dressing!', 
        'category': 'food' 
    },
    {
        'text': 'Why did the scarecrow win an award? Because he was out-standing in his field!', 
        'category': 'puns' 
    },
    {
        'text': 'What do you call a fake noodle? An impasta!', 
        'category': 'food' 
    },
    {
        'text': 'What do you call a belt made out of watches? A waist of time!', 
        'category': 'puns' 
    },
    {
        'text': 'Why did the math book look sad? Because it had too many problems!', 
        'category': 'math' 
    },
    {
        'text': 'Why did the gym close down? It just didn\'t work out!', 
        'category': 'exercise' 
    },
    {
        'text': 'Why don\'t scientists trust atoms? Because they make up everything!', 
        'category': 'science' 
    },
    {
        'text': 'What do you call a fake noodle? An impasta!', 
        'category': 'food' 
    },
    {
        'text': 'Why did the chicken cross the playground? To get to the other slide!', 
        'category': 'kids' 
    },
    {
        'text': 'Why did the frog call his insurance company? He had a jump in his car!', 
        'category': 'puns' 
    }

] 
# Load the BERT tokenizer and model 
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased') 
model = AutoModel.from_pretrained('bert-base-uncased') 

# Generate embeddings for the jokes using BERT 
for joke in jokes: 
    text = joke['text'] 
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True) 
    with torch.no_grad(): 
        output = model(**inputs).last_hidden_state.mean(dim=1).squeeze(0).numpy() 
        joke['embedding'] = output.tolist() 

# Index the jokes in Elasticsearch 
for joke in jokes: 
    es.index(index='jokes-index', body=joke) 

# Define the query vector 
# Define a query text and convert it to a dense vector using BERT
query = "What do you get when you cross a snowman and a shark?"
inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True)
with torch.no_grad():
    output = model(**inputs).last_hidden_state.mean(dim=1).squeeze(0).numpy()
query_vector = output

# Define the Elasticsearch KNN search 
search = {
    "knn": {
        "field": "embedding",
        "query_vector": query_vector.tolist(),
        "k": 3,
        "num_candidates": 100
    },
    "fields": [ "text" ]
}

# Perform the KNN search and print the results 
response = es.search(index='jokes-index', body=search)
for hit in response['hits']['hits']:
    print(f"Joke: {hit['_source']['text']}")


Joke: What do you call a fake noodle? An impasta!
Joke: What do you call a fake noodle? An impasta!
Joke: What did the cat say when he lost all his money? I am paw.
